In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import time

# 1. 自适应模型训练
def train_adaptive_model(fatigue_data):
    # 假设数据集的列名为 '疲劳应力'
    features = fatigue_data.drop('循环应力', axis=1)  # 假设'疲劳应力'列为目标
    target = fatigue_data['循环应力']  # 将目标列赋值为'疲劳应力'

    # 归一化处理
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)

    # 使用不同的模型进行训练
    models = {
        "SVR": SVR(),
        "RFR": RandomForestRegressor(random_state=42),
        "GBR": GradientBoostingRegressor(),
        "DTR": DecisionTreeRegressor(random_state=42)
    }

    # 选择最优模型
    best_model = None
    best_score = float('inf')
    for model_name, model in models.items():
        model.fit(features_scaled, target)
        pred = model.predict(features_scaled)
        score = mean_squared_error(target, pred)  # 使用均方误差来评估模型性能
        if score < best_score:
            best_score = score
            best_model = model
    
    # 返回最优模型和归一化Scaler
    return best_model, scaler

# 2. 设定阈值，并分类样本
def set_threshold_and_classify(fatigue_data, model, scaler):
    features = fatigue_data.drop('循环应力', axis=1)  # 去除目标列
    features_scaled = scaler.transform(features)  # 归一化处理
    
    predicted_y = model.predict(features_scaled)  # 使用最优模型进行预测

    # 计算误差 e1
    true_labels = fatigue_data['循环应力']
    error = np.abs(true_labels - predicted_y) / true_labels * 100  # 计算误差 e1
    fatigue_data['误差'] = error  # 将误差添加到数据中

    # 设置阈值 e1
    threshold = 5  # 可以调整阈值
    class1_data = fatigue_data[fatigue_data['误差'] > threshold]
    class2_data = fatigue_data[fatigue_data['误差'] <= threshold]

    # 将不满足阈值的样本放到新的表格中，为1
    class1_data['类别'] = 1
    # 将满足阈值的样本放到新的表格中，为2
    class2_data['类别'] = 2

    return class1_data, class2_data

# 3. 迁移学习和成分预测
def predict_composition_from_class2(class2_data, composition_model, scaler):
    # 提取class2中的特征
    features_class2 = class2_data[['固溶处理温度', '固溶处理时间', '稳定时效温度', '稳定时效时间', '时效温度', 
                                   '时效时间', '温度', '总应变幅', '弹性应变幅', '塑形应变幅', '循环应力']]
    
    # 对这些特征进行归一化处理
    features_class2_scaled = scaler.transform(features_class2)
    
    # 进行成分预测
    predicted_compositions = composition_model.predict(features_class2_scaled)
    class2_data['预测成分'] = predicted_compositions  # 将预测的成分加入到数据中
    
    return class2_data

# 4. 主优化循环函数
def optimization_loop():
    # 读取疲劳验证数据
    fatigue_data = pd.read_excel('疲劳验证.xlsx')  # 读取疲劳验证数据
    high_temp_alloy_data = pd.read_excel('高温合金数据库.xlsx')  # 读取高温合金数据库

    # 1. 使用自适应模型进行建模
    best_model, scaler = train_adaptive_model(fatigue_data)

    # 2. 设定阈值，并分类样本
    class1_data, class2_data = set_threshold_and_classify(fatigue_data, best_model, scaler)

    # 3. 迁移学习并进行成分预测
    composition_model = RandomForestRegressor(random_state=42)
    composition_model.fit(class2_data[['固溶处理温度', '固溶处理时间', '稳定时效温度', '稳定时效时间', '时效温度', 
                                       '时效时间', '温度', '总应变幅', '弹性应变幅', '塑形应变幅', '循环应力']],
                          class2_data['Ni'])  # 这里假设使用Ni作为目标进行训练（可以根据实际情况调整）
    
    # 将成分预测结果与工艺特征整合
    class2_data = predict_composition_from_class2(class2_data, composition_model, scaler)
    
    # 4. 生成新的表格，更新为'整合.xlsx'
    output_file = f"整合_{int(time.time())}.xlsx"  # 可以根据阈值或时间动态生成文件名
    class2_data.to_excel(output_file, index=False)

    # 5. 如果满足阈值条件，则结束循环，否则继续
    mean_error = class2_data['误差'].mean()
    print(f"当前循环的平均误差为: {mean_error}%")
    
    if mean_error < 5:  # 阈值e1 < 5时，停止循环
        print("满足阈值条件，循环结束。")
    else:
        print("误差过大，继续进行下一轮循环。")
        optimization_loop()  # 继续下一轮循环

# 运行优化循环
optimization_loop()


KeyError: "['疲劳应力'] not found in axis"